In [1]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import sys
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# required to enable non-blocking interaction:
%gui qt5

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import CapturedException

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch


from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme
from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows


from pyphocorehelpers.print_helpers import CapturedException
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler



BATCH_DATE_TO_USE = '2023-08-24' # used for filenames throught the notebook

Automatic pdb calling has been turned OFF
build_module_logger(module_name="Spike3D.pipeline"):
	 Module logger com.PhoHale.Spike3D.pipeline has file logging enabled and will log to EXTERNAL\TESTING\Logging\debug_com.PhoHale.Spike3D.pipeline.log


In [2]:
# active_global_batch_result_filename='global_batch_result_2023-07-12.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-07.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-07_extra.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-20.pkl'
# active_global_batch_result_filename='global_batch_result_2023-07-21_Greatlakes.pkl'
active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'
debug_print = False
known_global_data_root_parent_paths = [Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/nfs/turbo/umms-kdiba/Data')]
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

# try to load an existing batch result:
# with set_posix_windows():
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename,
						skip_root_path_conversion=False, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch

batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     # display(batch_progress_df)
#     # display(good_only_batch_progress_df)
#     display(batch_progress_df)

Loading loaded session pickle file results : W:\Data\global_batch_result_2023-08-24.pkl... done.
no difference between provided and internal paths.


,format_name,animal,exper_name,session_name,session_datetime,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
0,kdiba,gor01,one,2006-6-07_11-26-53,2006-06-07 11:26:53,kdiba_gor01_one_2006-6-07_11-26-53,W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\out...,W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\loa...,W:\Data\KDIBA\gor01\one\2006-6-07_11-26-53\rip...,False,True
1,kdiba,gor01,one,2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba_gor01_one_2006-6-08_14-26-15,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\out...,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loa...,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\rip...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,2006-06-09 01:22:43,kdiba_gor01_one_2006-6-09_1-22-43,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\outp...,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\load...,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\ripp...,True,True
3,kdiba,gor01,one,2006-6-09_3-23-37,2006-06-09 03:23:37,kdiba_gor01_one_2006-6-09_3-23-37,W:\Data\KDIBA\gor01\one\2006-6-09_3-23-37,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,W:\Data\KDIBA\gor01\one\2006-6-09_3-23-37\ripp...,False,True
4,kdiba,gor01,one,2006-6-12_15-55-31,2006-06-12 15:55:31,kdiba_gor01_one_2006-6-12_15-55-31,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\out...,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\loa...,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\rip...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,kdiba,pin01,one,fet11-04_21-20-3,2009-11-04 21:20:03,kdiba_pin01_one_fet11-04_21-20-3,W:\Data\KDIBA\pin01\one\fet11-04_21-20-3,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,W:\Data\KDIBA\pin01\one\fet11-04_21-20-3\loade...,W:\Data\KDIBA\pin01\one\fet11-04_21-20-3\fet11...,False,True
68,kdiba,pin01,one,redundant,NaT,kdiba_pin01_one_redundant,W:\Data\KDIBA\pin01\one\redundant,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,,False,False
69,kdiba,pin01,one,showclus,NaT,kdiba_pin01_one_showclus,W:\Data\KDIBA\pin01\one\showclus,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,,False,False
70,kdiba,pin01,one,sleep,NaT,kdiba_pin01_one_sleep,W:\Data\KDIBA\pin01\one\sleep,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,,,,False,False


# Run Batch Executions/Computations

In [3]:
# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

included_session_batch_progress_df = batch_progress_df[np.isin(batch_progress_df['context'].values, included_session_contexts)]
included_session_batch_progress_df

,format_name,animal,exper_name,session_name,session_datetime,context,basedirs,status,errors,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
1,kdiba,gor01,one,2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba_gor01_one_2006-6-08_14-26-15,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\out...,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\loa...,W:\Data\KDIBA\gor01\one\2006-6-08_14-26-15\rip...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,2006-06-09 01:22:43,kdiba_gor01_one_2006-6-09_1-22-43,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\outp...,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\load...,W:\Data\KDIBA\gor01\one\2006-6-09_1-22-43\ripp...,True,True
4,kdiba,gor01,one,2006-6-12_15-55-31,2006-06-12 15:55:31,kdiba_gor01_one_2006-6-12_15-55-31,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\out...,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\loa...,W:\Data\KDIBA\gor01\one\2006-6-12_15-55-31\rip...,True,True
6,kdiba,gor01,two,2006-6-07_16-40-19,2006-06-07 16:40:19,kdiba_gor01_two_2006-6-07_16-40-19,W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\out...,W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\loa...,W:\Data\KDIBA\gor01\two\2006-6-07_16-40-19\rip...,True,True
8,kdiba,gor01,two,2006-6-08_21-16-25,2006-06-08 21:16:25,kdiba_gor01_two_2006-6-08_21-16-25,W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\out...,W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\loa...,W:\Data\KDIBA\gor01\two\2006-6-08_21-16-25\rip...,True,True
9,kdiba,gor01,two,2006-6-09_22-24-40,2006-06-09 22:24:40,kdiba_gor01_two_2006-6-09_22-24-40,W:\Data\KDIBA\gor01\two\2006-6-09_22-24-40,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\two\2006-6-09_22-24-40\out...,W:\Data\KDIBA\gor01\two\2006-6-09_22-24-40\loa...,W:\Data\KDIBA\gor01\two\2006-6-09_22-24-40\rip...,True,True
10,kdiba,gor01,two,2006-6-12_16-53-46,2006-06-12 16:53:46,kdiba_gor01_two_2006-6-12_16-53-46,W:\Data\KDIBA\gor01\two\2006-6-12_16-53-46,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\gor01\two\2006-6-12_16-53-46\out...,W:\Data\KDIBA\gor01\two\2006-6-12_16-53-46\loa...,W:\Data\KDIBA\gor01\two\2006-6-12_16-53-46\rip...,True,True
12,kdiba,vvp01,one,2006-4-09_17-29-30,2006-04-09 17:29:30,kdiba_vvp01_one_2006-4-09_17-29-30,W:\Data\KDIBA\vvp01\one\2006-4-09_17-29-30,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\vvp01\one\2006-4-09_17-29-30\out...,W:\Data\KDIBA\vvp01\one\2006-4-09_17-29-30\loa...,W:\Data\KDIBA\vvp01\one\2006-4-09_17-29-30\rip...,True,True
13,kdiba,vvp01,one,2006-4-10_12-25-50,2006-04-10 12:25:50,kdiba_vvp01_one_2006-4-10_12-25-50,W:\Data\KDIBA\vvp01\one\2006-4-10_12-25-50,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\vvp01\one\2006-4-10_12-25-50\out...,W:\Data\KDIBA\vvp01\one\2006-4-10_12-25-50\loa...,W:\Data\KDIBA\vvp01\one\2006-4-10_12-25-50\rip...,True,True
31,kdiba,vvp01,two,2006-4-09_16-40-54,2006-04-09 16:40:54,kdiba_vvp01_two_2006-4-09_16-40-54,W:\Data\KDIBA\vvp01\two\2006-4-09_16-40-54,SessionBatchProgress.NOT_STARTED,None,0,0,0,0,False,W:\Data\KDIBA\vvp01\two\2006-4-09_16-40-54\out...,W:\Data\KDIBA\vvp01\two\2006-4-09_16-40-54\loa...,W:\Data\KDIBA\vvp01\two\2006-4-09_16-40-54\rip...,True,True


# Execute Batch

In [ ]:

@define(slots=False, repr=False)
class HDFSpecificBatchSessionCompletionHandler(BatchSessionCompletionHandler):
    """ handles completion of a single session's batch processing. 

    Allows accumulating results across sessions and runs.

    
    Usage:
        from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler
        
    """

    # Completion Result object returned from callback ____________________________________________________________________ #
 
    # Cross-session Results:
    across_sessions_instantaneous_fr_dict: dict = Factory(dict) # Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation

    ## Main function that's called with the complete pipeline:
    def on_complete_success_execution_session(self, global_data_root_parent_path, curr_session_context, curr_session_basedir, curr_active_pipeline) -> PipelineCompletionResult:
        """ called when the execute_session completes like:
            `post_run_callback_fn_output = post_run_callback_fn(curr_session_context, curr_session_basedir, curr_active_pipeline)`
            
            Meant to be assigned like:
            , post_run_callback_fn=_on_complete_success_execution_session
            
            Captures nothing.
            
            from Spike3D.scripts.run_BatchAnalysis import _on_complete_success_execution_session
            
            
            LOGIC: really we want to recompute global whenever local is recomputed.
            
            
        """
        print(f'HDFProcessing.on_complete_success_execution_session(curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}, ...)')
        # print(f'curr_session_context: {curr_session_context}, curr_session_basedir: {str(curr_session_basedir)}')
        long_epoch_name, short_epoch_name, global_epoch_name = curr_active_pipeline.find_LongShortGlobal_epoch_names()
        # long_session, short_session, global_session = [curr_active_pipeline.filtered_sessions[an_epoch_name] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
        # long_results, short_results, global_results = [curr_active_pipeline.computation_results[an_epoch_name]['computed_data'] for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]

        # Get existing laps from session:
        long_laps, short_laps, global_laps = [curr_active_pipeline.filtered_sessions[an_epoch_name].laps.as_epoch_obj() for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
        long_replays, short_replays, global_replays = [Epoch(curr_active_pipeline.filtered_sessions[an_epoch_name].replay.epochs.get_valid_df()) for an_epoch_name in [long_epoch_name, short_epoch_name, global_epoch_name]]
        # short_laps.n_epochs: 40, n_long_laps.n_epochs: 40
        # short_replays.n_epochs: 6, long_replays.n_epochs: 8
        if self.debug_print:
            print(f'short_laps.n_epochs: {short_laps.n_epochs}, n_long_laps.n_epochs: {long_laps.n_epochs}')
            print(f'short_replays.n_epochs: {short_replays.n_epochs}, long_replays.n_epochs: {long_replays.n_epochs}')

        # ## Post Compute Validate 2023-05-16:
        try:
            was_updated = self.post_compute_validate(curr_active_pipeline)
        except Exception as e:
            exception_info = sys.exc_info()
            an_err = CapturedException(e, exception_info)
            print(f'self.post_compute_validate(...) failed with exception: {an_err}')
            raise 

        delta_since_last_compute: timedelta = curr_active_pipeline.get_time_since_last_computation()
        print(f'\t time since last computation: {delta_since_last_compute}')
        
        ## Save the pipeline since that's disabled by default now:
        try:
            curr_active_pipeline.save_pipeline(saving_mode=self.saving_mode, active_pickle_filename=self.override_session_computation_results_pickle_filename) # AttributeError: 'PfND_TimeDependent' object has no attribute '_included_thresh_neurons_indx'
        except Exception as e:
            ## TODO: catch/log saving error and indicate that it isn't saved.
            exception_info = sys.exc_info()
            e = CapturedException(e, exception_info)
            print(f'ERROR SAVING PIPELINE for curr_session_context: {curr_session_context}. error: {e}')


        ## GLOBAL FUNCTION:
        if self.force_reload_all and (not self.force_global_recompute):
            print(f'WARNING: self.force_global_recompute was False but self.force_reload_all was true. The global properties must be recomputed when the local functions change, so self.force_global_recompute will be set to True and computation will continue.')
            self.force_global_recompute = True
            
        if was_updated and (not self.force_global_recompute):
            print(f'WARNING: self.force_global_recompute was False but pipeline was_updated. The global properties must be recomputed when the local functions change, so self.force_global_recompute will be set to True and computation will continue.')
            self.force_global_recompute = True


        ## GLOBAL FUNCTION:
        if self.force_reload_all and (not self.force_global_recompute):
            print(f'WARNING: self.force_global_recompute was False but self.force_reload_all was true. The global properties must be recomputed when the local functions change, so self.force_global_recompute will be set to True and computation will continue.')
            self.force_global_recompute = True
            
        if was_updated and (not self.force_global_recompute):
            print(f'WARNING: self.force_global_recompute was False but pipeline was_updated. The global properties must be recomputed when the local functions change, so self.force_global_recompute will be set to True and computation will continue.')
            self.force_global_recompute = True

        self.try_compute_global_computations_if_needed(curr_active_pipeline, curr_session_context=curr_session_context)

        ### Aggregate Outputs specific computations:

        ## Get some more interesting session properties:
        
        delta_since_last_compute: timedelta = curr_active_pipeline.get_time_since_last_computation()
        
        print(f'\t time since last computation: {delta_since_last_compute}')

        # Export the pipeline's HDF5:
        hdf5_output_path, hdf5_output_err = self.try_export_pipeline_hdf5_if_needed(curr_active_pipeline=curr_active_pipeline, curr_session_context=curr_session_context)
        
        # ## Specify the output file:
        # common_file_path = Path('output/active_across_session_scatter_plot_results.h5')
        # print(f'common_file_path: {common_file_path}')
        # InstantaneousFiringRatesDataframeAccessor.add_results_to_inst_fr_results_table(curr_active_pipeline, common_file_path, file_mode='a')

        across_session_results_extended_dict = {}
            
        return PipelineCompletionResult(long_epoch_name=long_epoch_name, long_laps=long_laps, long_replays=long_replays,
                                           short_epoch_name=short_epoch_name, short_laps=short_laps, short_replays=short_replays,
                                           delta_since_last_compute=delta_since_last_compute,
                                           outputs_local={'pkl': curr_active_pipeline.pickle_path},
                                            outputs_global={'pkl': curr_active_pipeline.global_computation_results_pickle_path, 'hdf5': hdf5_output_path},
                                            across_session_results=across_session_results_extended_dict)
                                          


In [ ]:
multiprocessing_kwargs = dict(use_multiprocessing=False, num_processes=1)
# multiprocessing_kwargs = dict(use_multiprocessing=True, num_processes=2)

enable_full_pipeline_in_ram = False
# enable_full_pipeline_in_ram = True

# Whether to output figures:
should_perform_figure_generation_to_file=False
# should_perform_figure_generation_to_file=True

## Included Session Contexts:
# included_session_contexts = batch_progress_df[np.logical_and(batch_progress_df['has_user_replay_annotations'], batch_progress_df['is_ready'])]['context'].to_numpy().tolist()

# Only require sessions to have replay annotations:
# included_session_contexts = batch_progress_df[batch_progress_df['has_user_replay_annotations']]['context'].to_numpy().tolist()

# included_session_contexts = batch_progress_df['context'].to_numpy().tolist()[:4] # Only get the first 6
## Limit the contexts to run to the last N:
# included_session_contexts = included_session_contexts[3:6]

# ALL
included_session_contexts = included_session_contexts

# ## No filtering the sessions:
# included_session_contexts = None

if included_session_contexts is not None:
    print(f'len(included_session_contexts): {len(included_session_contexts)}')
else:
    print(f'included_session_contexts is None so all session contexts will be included.')

# included_session_contexts


# No Reloading
result_handler = BatchSessionCompletionHandler(force_reload_all=False,
                                                session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=False),
                                                global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=False, should_save=False),
                                                should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
                                                enable_full_pipeline_in_ram=enable_full_pipeline_in_ram,
                                                **multiprocessing_kwargs)


# # Forced Reloading:
# result_handler = BatchSessionCompletionHandler(force_reload_all=True,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=True),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=True),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE, force_global_recompute=True,
#                                                 **multiprocessing_kwargs)


active_post_run_callback_fn = result_handler.on_complete_success_execution_session
# active_post_run_callback_fn = _temp_on_complete_success_execution_session


## Execute with the custom arguments.
active_computation_functions_name_includelist=['_perform_baseline_placefield_computation',
                                        # '_perform_time_dependent_placefield_computation',
                                        '_perform_extended_statistics_computation',
                                        '_perform_position_decoding_computation', 
                                        '_perform_firing_rate_trends_computation',
                                        '_perform_pf_find_ratemap_peaks_computation',
                                        # '_perform_time_dependent_pf_sequential_surprise_computation'
                                        '_perform_two_step_position_decoding_computation',
                                        # '_perform_recursive_latent_placefield_decoding'
                                    ]
# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']
global_batch_run.execute_all(force_reload=result_handler.force_reload_all, saving_mode=result_handler.saving_mode, skip_extended_batch_computations=True, post_run_callback_fn=active_post_run_callback_fn,
                             fail_on_exception=True, included_session_contexts=included_session_contexts,
                                                                                        **{'computation_functions_name_includelist': active_computation_functions_name_includelist,
                                                                                            'active_session_computation_configs': None,
                                                                                            'allow_processing_previously_completed': True}, **multiprocessing_kwargs) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

# Across Sessions After Batching Complete

In [6]:
included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_global_computation_h5_paths = [a_dir.joinpath('output','global_computations.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']] 

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import H5FileReference, H5ExternalLinkBuilder, ExternallyReferencedItem
from neuropy.utils.mixins.HDF5_representable import HDF_Converter

session_short_names: List[str] = [a_ctxt.get_description(separator='_') for a_ctxt in included_session_contexts] # 'kdiba.gor01.one.2006-6-08_14-26-15'
session_group_keys: List[str] = [("/" + a_ctxt.get_description(separator="/", include_property_names=False)) for a_ctxt in included_session_contexts] # 'kdiba/gor01/one/2006-6-08_14-26-15'
# Particular Table Keys:
neuron_replay_stats_df_table_keys = [f"{session_group_key}/global_computations/jonathan_fr_analysis/neuron_replay_stats_df/table" for session_group_key in session_group_keys]
neuron_identities_table_keys = [f"{session_group_key}/neuron_identities/table" for session_group_key in session_group_keys]
long_short_fr_indicies_analysis_table_keys = [f"{session_group_key}/global_computations/long_short_fr_indicies_analysis/table" for session_group_key in session_group_keys]

a_loader = H5ExternalLinkBuilder.init_from_file_lists(file_list=included_h5_paths, table_key_list=[f"{session_group_key}/neuron_identities/table" for session_group_key in session_group_keys], short_name_list=session_short_names)

In [ ]:
included_h5_paths

In [ ]:

# neuron_identities_table_keys = [ExternallyReferencedItem(f"{session_group_key}/neuron_identities/table", '/neuron_identities') for session_group_key in session_group_keys]
destination_file_path, external_file_links = a_loader.build_linking_results('output/test_linking_file.h5', referential_group_key='neuron_identities', table_key_list=neuron_identities_table_keys, fail_on_exception=False)
external_file_links


In [ ]:

destination_file_path, external_file_links = a_loader.build_linking_results('output/test_linking_file_new.h5', referential_group_key='long_short_fr_indicies_analysis', table_key_list=long_short_fr_indicies_analysis_table_keys, fail_on_exception=False)
external_file_links


In [ ]:
# destination_file_path, external_file_links = a_loader.build_linking_results('output/test_linking_file_new.h5', referential_group_key='neuron_replay_stats_df', table_key_list=neuron_replay_stats_df_table_keys, fail_on_exception=False)
# external_file_links

In [7]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionTables

_out_table = AcrossSessionTables. build_neuron_replay_stats_table(included_session_contexts, included_h5_paths)
_out_table

concatenating dataframes from 15 of 15 files


,format_name,animal,exper_name,session_name,session_datetime,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,...,long_mean,short_mean,mean_diff,num_replays,long_num_replays,short_num_replays,neuron_type,aclu,session_uid,neuron_uid
0,kdiba,gor01,one,2006-6-08_14-26-15,2006-06-08 14:26:15,212.160000,NaN,LEFT_ONLY,0.419960,0.114724,...,4.773420,NaN,NaN,31,31,0,pyr,2,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|2
1,kdiba,gor01,one,2006-6-08_14-26-15,2006-06-08 14:26:15,136.160000,198.160000,SHARED,0.275226,1.227759,...,4.986733,5.454890,0.468157,71,19,52,pyr,3,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|3
2,kdiba,gor01,one,2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,178.160000,RIGHT_ONLY,0.117397,0.834271,...,NaN,7.565518,NaN,67,0,67,pyr,4,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|4
3,kdiba,gor01,one,2006-6-08_14-26-15,2006-06-08 14:26:15,98.160000,202.160000,SHARED,2.090985,1.844885,...,8.092135,7.583226,-0.508909,163,91,72,pyr,5,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|5
4,kdiba,gor01,one,2006-6-08_14-26-15,2006-06-08 14:26:15,NaN,NaN,SHARED,26.129319,12.892497,...,68.830796,27.813785,-41.017012,495,279,216,intr,6,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
882,kdiba,pin01,one,fet11-01_12-58-54,2009-11-01 12:58:54,179.403791,177.403791,SHARED,0.892852,1.027642,...,6.178167,7.307386,1.129219,91,52,39,pyr,28,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28
883,kdiba,pin01,one,fet11-01_12-58-54,2009-11-01 12:58:54,59.403791,107.403791,SHARED,0.837484,0.766268,...,5.987154,6.376664,0.389510,164,121,43,pyr,29,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|29
884,kdiba,pin01,one,fet11-01_12-58-54,2009-11-01 12:58:54,119.403791,127.403791,SHARED,0.712330,0.577649,...,6.552028,6.036531,-0.515497,88,56,32,pyr,30,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|30
885,kdiba,pin01,one,fet11-01_12-58-54,2009-11-01 12:58:54,217.403791,33.403791,SHARED,0.902182,1.016473,...,5.656711,5.564682,-0.092028,115,80,35,pyr,31,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|31


In [ ]:
# How many of each track_membership type in each session?
# _gdf = _out_table.groupby(['session_uid', 'track_membership']).agg(index_count=('neuron_uid', 'count')).reset_index()
# _gdf.to_clipboard(excel=True, index=False)
# Options
session_identifier_key: str = 'session_name'
# session_identifier_key: str = 'session_datetime'

## Count of the fields of interest using .value_counts(...)
_out_value_counts: pd.Series = _out_table.value_counts(subset=['format_name', 'animal', session_identifier_key,'track_membership'], normalize=False, sort=False, ascending=True, dropna=True)
## !IMPORTANT! Converting a `df.value_counts(...)` result to an explicit pd.DataFrame
_out_value_counts_df = _out_value_counts.reset_index()
_out_value_counts_df.columns = ['format_name', 'animal', session_identifier_key, 'track_membership', 'count']
_out_value_counts_df

In [ ]:
_out_value_counts_df.to_clipboard(sep=',', excel=True, index=False)

### Alluvial Flow Diagrams with `floweaver`

In [ ]:
import floweaver as fw


df = deepcopy(_out_value_counts_df)

nodes = {
    'animal': fw.ProcessGroup(['gor01', 'pin01', 'vvp01'], title='Animal'),
    'track': fw.ProcessGroup(['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED'], title='TrackXclusivity'),
}

ordering = [
    ['animal'],  # put "animal" on the left...
    ['track'],   # ... and "track" on the right.
]

bundles = [
    fw.Bundle('animal', 'track'),
]

partitions = [
    fw.Partition.Simple('process', ['gor01', 'pin01', 'vvp01']),
    fw.Partition.Simple('process', ['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED']),
]

nodes['animal'].partition = partitions[0]
nodes['track'].partition = partitions[1]
sdd = fw.SankeyDefinition(nodes, bundles, ordering, flow_partition=fw.Partition.Simple('process', []))

flow_data = df.groupby(['animal', 'track_membership'])['count'].sum().reset_index()
flow_data.columns = ['source', 'target', 'value']
fw.weave(sdd, flow_data, palette='Set2_8').to_widget().auto_save_png('alluvial_diagram.png')


#### Three-item flow:

In [ ]:

import floweaver as fw



def plot_alluvial_diagram(df: pd.DataFrame, session_identifier_key:str='session_datetime'):
	""" 2023-08-24 - Renders a flow diagram showing the breakdown of LxC, SxC, Shared neurons in each session for each animal. 

	Usage:
		alluvial_widget, sdd, flow_data = plot_alluvial_diagram(_out_value_counts_df, session_identifier_key=session_identifier_key)
		alluvial_widget

	"""
	df = df.copy()
	
	## Somehow I want to get a 'type' key through this process and into the final flow_data DataFrame so that I can color based on it as mentioned in this demo: https://notebooks.gesis.org/binder/jupyter/user/ricklupton-floweaver-586o9ybx/notebooks/docs/tutorials/quickstart.ipynb#
	
	# Define the flow from "animal" to "session_datetime"
	flow_animal_to_session = df.groupby(['animal', session_identifier_key])['count'].sum().reset_index()
	flow_animal_to_session.columns = ['source', 'target', 'value']

	# Define the flow from "session_datetime" to "track_membership"
	flow_session_to_track = df.groupby([session_identifier_key, 'track_membership'])['count'].sum().reset_index()
	flow_session_to_track.columns = ['source', 'target', 'value']

	# Combine the two flows
	flow_data: pd.DataFrame = pd.concat([flow_animal_to_session, flow_session_to_track], ignore_index=True)

	# Define the unique session identifiers:
	unique_session_identifiers = df[session_identifier_key].unique().tolist()

	nodes = {
		'animal': fw.ProcessGroup(['gor01', 'pin01', 'vvp01'], title='Animal'),
		'session': fw.ProcessGroup(unique_session_identifiers, title=session_identifier_key),  # New node for session_datetime
		'track': fw.ProcessGroup(['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED'], title='TrackXclusivity'),
	}

	ordering = [
		['animal'],
		['session'],  # Include session_datetime node
		['track'],
	]

	bundles = [
		fw.Bundle('animal', 'session'),  # Bundle from animal to session
		fw.Bundle('session', 'track'),   # Bundle from session to track
	]

	partitions = [
		fw.Partition.Simple('process', ['gor01', 'pin01', 'vvp01']),
		fw.Partition.Simple('process', unique_session_identifiers),  # Partition for session_datetime
		fw.Partition.Simple('process', ['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED']),
	]

	nodes['animal'].partition = partitions[0]
	nodes['session'].partition = partitions[1]  # Assign partition for session_datetime
	nodes['track'].partition = partitions[2]
	
	flow_partition = fw.Partition.Simple('process', [])
	#flow_partition = fw.Partition.Simple('process', ['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED']) # partitions[2]
	sdd = fw.SankeyDefinition(nodes, bundles, ordering, flow_partition=flow_partition) # 

	## Weave the plot:
	_out_widget = fw.weave(sdd, flow_data).to_widget().auto_save_png('alluvial_diagram_with_session.png')

	return _out_widget, sdd, flow_data



alluvial_widget, sdd, flow_data = plot_alluvial_diagram(_out_value_counts_df, session_identifier_key=session_identifier_key)
alluvial_widget

In [ ]:
_out_table_long_short_fr_indicies_analysis = a_loader.load_and_consolidate(table_key_list=long_short_fr_indicies_analysis_table_keys, fail_on_exception=False)
_out_table_long_short_fr_indicies_analysis = HDF_Converter.general_post_load_restore_table_as_needed(_out_table_long_short_fr_indicies_analysis)
_out_table_long_short_fr_indicies_analysis

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=_out_table_long_short_fr_indicies_analysis, num_sessions=0)


In [ ]:
# TODO: More robust and automatic attempt to restore native column types:
# _out_table = HDF_Converter.HDFConvertableEnum.restore_hdf_dataframe_column_original_type(_out_table, column_name='track_membership') # AttributeError: 'int' object has no attribute 'lower'
# SplitPartitionMembership.restore_hdf_dataframe_column_original_type(_out_table, column_name='track_membership')
# _out_table = HDF_Converter.HDFConvertableEnum.restore_hdf_dataframe_column_original_type(_out_table, column_name='neuron_type')
# _out_table["neuron_type"] = _out_table["neuron_type"].apply(lambda x: NeuronType.from_hdf_coding_string(x)).astype(object) #.astype(str) # interestingly this leaves the dtype of this column as 'category' still, but _spikes_df["cell_type"].to_numpy() returns the correct array of objects... this is better than it started before saving, but not the same. 
_out_table

In [ ]:
_out_table.groupby(['session_uid', 'track_membership', 'neuron_type']).agg(neuron_uid_count=('neuron_uid', 'count')).reset_index()

In [ ]:
global_batch_run

In [ ]:
included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_global_computation_h5_paths = [a_dir.joinpath('output','global_computations.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']] 
included_h5_paths

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import check_output_h5_files

df = check_output_h5_files(included_file_paths=included_h5_paths)
df

In [ ]:

from pyphocorehelpers.Filesystem.metadata_helpers import 
[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_h5_paths]
[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_h5_paths]


def get_file_metadata(a_file):
    FilesystemMetadata.get_last_modified_time(a_file)

pd.DataFrame({'session_context':included_session_batch_progress_df['context'],
              'h5_paths': included_h5_paths,
              'modified_time':[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_h5_paths],
              'global_result_modified_time':[FilesystemMetadata.get_last_modified_time(a_file) for a_file in included_global_computation_h5_paths],
             })

In [ ]:
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import LoadedObjectPersistanceState

In [ ]:
batch_progress_df[np.isin(batch_progress_df['context'].values, included_session_contexts)]

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df

In [ ]:
## 2023-08-10
# Add output modes for .pkl and .h5 files similar to figures. For GreatLakes Batch runs, allow output to the run folder.

# like the figures, file naming conventions should change for the different types.


In [ ]:
import seaborn as sns
from pyphocorehelpers.indexing_helpers import partition, safe_pandas_get_group
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor, AcrossSessionsVisualizations
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import SingleBarResult

common_file_path = Path('output/active_across_session_scatter_plot_results.h5')
print(f'common_file_path: {common_file_path}')

# InstantaneousSpikeRateGroupsComputation, : pd.DataFrame
_shell_obj, loaded_result_df = InstantaneousFiringRatesDataframeAccessor.load_and_prepare_for_plot(common_file_path)
# Perform the actual plotting:
AcrossSessionsVisualizations.across_sessions_bar_graphs(_shell_obj, num_sessions=13, save_figure=False, enable_tiny_point_labels=False, enable_hover_labels=False)

In [ ]:
# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
sessions_with_results

In [ ]:
session_identifiers = included_session_contexts


In [ ]:
global_batch_run

In [ ]:
# Save to pickle:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary

In [ ]:
# Combine across .h5 files:
import tables as tb
import h5py
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import H5ExternalLinkBuilder

# f'/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table'

session_identifiers, pkl_output_paths, hdf5_output_paths

file_path = f'output/test_external_links.h5'
a_global_computations_group_key: str = f"{session_group_key}/global_computations"
with tb.open_file(file_path, mode='w') as f: # this mode='w' is correct because it should overwrite the previous file and not append to it.
    a_global_computations_group = f.create_group(session_group_key, 'global_computations', title='the result of computations that operate over many or all of the filters in the session.', createparents=True)
    an_external_link = f.create_external_link(f'', n

In [ ]:


from typing import List
from attrs import define, field, Factory
import pandas as pd
import tables as tb

    
session_group_keys: List[str] = [("/" + a_ctxt.get_description(separator="/", include_property_names=False)) for a_ctxt in session_identifiers] # 'kdiba/gor01/one/2006-6-08_14-26-15'
# session_uid: str = session_context.get_description(separator="|", include_property_names=False)
neuron_identities_table_keys = [f"{session_group_key}/neuron_identities/table" for session_group_key in session_group_keys]


a_loader = H5ExternalLinkBuilder(file_list=hdf5_output_paths, table_key_list=neuron_identities_table_keys)
a_loader

_out_table = a_loader.load_and_consolidate()
_out_table

## Extract output files from all completed sessions:

In [ ]:
# '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5'

# kdiba_vvp01_two_2006-4-10_12-58-3
# 	outputs_local ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle.pkl')}
# 	outputs_global ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results.pkl'), 'hdf5': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pipeline_results.h5')}

session_identifiers, pkl_output_paths, hdf5_output_paths = global_batch_run.build_output_files_lists()

In [ ]:
session_identifiers

In [ ]:
pkl_output_paths

In [ ]:
hdf5_output_paths

In [ ]:
# Save output filelist:


def save_filelist_to_text_file(hdf5_output_paths, filelist_path: Path):
    _out_string = '\n'.join([str(a_file) for a_file in hdf5_output_paths])
    print(f'{_out_string}')
    print(f'saving out to "{filelist_path}"...')
    with open(filelist_path, 'w') as f:
        f.write(_out_string)
    return _out_string, filelist_path


h5_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_HDF5_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_HDF5_savepath = save_filelist_to_text_file(hdf5_output_paths, h5_filelist_path)

pkls_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_pkls_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_pkls_savepath = save_filelist_to_text_file(pkl_output_paths, pkls_filelist_path)


In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent
# source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
source_parent_path = Path(r'/nfs/turbo/umms-kdiba/Data')
dest_parent_path = Path(r'/~/W/Data/')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_source = hdf5_output_paths
filelist_dest_paths = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest_paths

dest_Apogee_h5_filelist_path = global_data_root_parent_path.joinpath(f'dest_fileList_Apogee_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, dest_filelist_savepath = save_filelist_to_text_file(filelist_dest_paths, dest_Apogee_h5_filelist_path)

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from neuropy.core.epoch import Epoch

# Save to HDF5
suffix = f'{BATCH_DATE_TO_USE}'
## Build Pickle Path:
file_path = global_data_root_parent_path.joinpath(f'global_batch_output_{suffix}.h5').resolve()
file_path

In [ ]:
global_batch_run.to_hdf(file_path,'/')

In [ ]:
# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]

# list(global_batch_run.session_batch_outputs.keys())

# Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation

# good_session_batch_outputs = global_batch_run.session_batch_outputs

sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
good_session_batch_outputs = {a_ctxt:a_result for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}

for a_ctxt, a_result in good_session_batch_outputs.items():
    if a_result is not None:
        # a_good_result = a_result.__dict__.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
        a_good_result = a_result.across_session_results.get('inst_fr_comps', None)
        if a_good_result is not None:
            across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
            # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
            
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
across_sessions_instantaneous_fr_dict

In [ ]:
[a_ctxt.get_initialization_code_string() for a_ctxt in sessions_with_results]

# OLD

## 2023-07-14 - Load Saved across-sessions-data and testing Batch-computed inst_firing_rates:

In [ ]:
# from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import list_of_dicts_to_dict_of_lists
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
across_sessions_instantaneous_frs_list

In [ ]:
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation` 
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False)


In [ ]:
across_session_inst_fr_computation.LxC_scatter_props
across_session_inst_fr_computation.SxC_scatter_props

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, InstantaneousFiringRatesDataframeAccessor, InstantaneousSpikeRateGroupsComputation, trackMembershipTypesEnum, trackExclusiveToMembershipTypeDict, trackExclusiveToMembershipTypeReverseDict

## Specify the output file:
common_file_path = Path('output/test_across_session_scatter_plot_new.h5')
print(f'common_file_path: {common_file_path}')
InstantaneousFiringRatesDataframeAccessor.add_results_to_inst_fr_results_table(curr_active_pipeline, common_file_path, file_mode='a')

In [ ]:
# Build the unique scatter plot dictionaries:
across_session_contexts = list(across_sessions_instantaneous_fr_dict.keys())
unique_animals = IdentifyingContext.find_unique_values(across_session_contexts)['animal'] # {'gor01', 'pin01', 'vvp01'}
# Get number of animals to plot
marker_list = [(5, i) for i in np.arange(len(unique_animals))] # [(5, 0), (5, 1), (5, 2)]
scatter_props = [{'marker': mkr} for mkr in marker_list]  # Example, you should provide your own scatter properties
scatter_props_dict = dict(zip(unique_animals, scatter_props))
# {'pin01': {'marker': (5, 0)},
#  'gor01': {'marker': (5, 1)},
#  'vvp01': {'marker': (5, 2)}}
scatter_props_dict

# Pass a function that will return a set of kwargs for a given context
def _return_scatter_props_fn(ctxt: IdentifyingContext):
	""" captures `scatter_props_dict` """
	animal_id = str(ctxt.animal)
	return scatter_props_dict[animal_id]
	


In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation # the result loaded from the file
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
# _out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)
_out_fig_2.scatter_props_fn = _return_scatter_props_fn

In [ ]:
LxC_aclus = _out_fig_2.computation_result.LxC_aclus
SxC_aclus = _out_fig_2.computation_result.SxC_aclus

LxC_aclus

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureOutputManager, FigureOutputLocation, ContextToPathMode

registered_output_files = {}

def output_figure(final_context: IdentifyingContext, fig, write_vector_format:bool=False, write_png:bool=True, debug_print=True):
    """ outputs the figure using the provided context. """
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_and_write_to_file
    def register_output_file(output_path, output_metadata=None):
        """ registers a new output file for the pipeline """
        print(f'register_output_file(output_path: {output_path}, ...)')
        registered_output_files[output_path] = output_metadata or {}

    fig_out_man = FigureOutputManager(figure_output_location=FigureOutputLocation.DAILY_PROGRAMMATIC_OUTPUT_FOLDER, context_to_path_mode=ContextToPathMode.HIERARCHY_UNIQUE)
    active_out_figure_paths = build_and_write_to_file(fig, final_context, fig_out_man, write_vector_format=write_vector_format, write_png=write_png, register_output_file_fn=register_output_file)
    return active_out_figure_paths, final_context


# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = output_figure

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

### Single Session testing:


In [ ]:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_includelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_includelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

## Get good sessions for use in the specific session processing notebook:

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

In [ ]:
run["good_sessions_list"].extend(full_good_ready_to_process_sessions)

In [ ]:
run.stop()
project.stop()

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

In [ ]:
good_only_batch_progress_df

In [ ]:
from datetime import datetime

# datetime object containing current date and time
save_time = datetime.now()
 
print("save_time =", save_time)

# dd/mm/YY H:M:S
dt_string = save_time.strftime("%Y-%m-%d_%I-%M%p")
print("date and time =", dt_string)

In [ ]:
## Get output file paths:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'outputs/global_computation_results.pkl'

full_good_ready_to_process_session_paths = list(good_only_batch_progress_df['basedirs'].to_numpy())
session_paths_output_folders = [sess_path.joinpath('outputs').resolve() for sess_path in full_good_ready_to_process_session_paths]

completed_pipeline_file_paths = [sess_path.joinpath(completed_pipeline_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths = [sess_path.joinpath(completed_global_computations_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths

In [ ]:
# Countable Additivity 
# Any countable collections of points is size 0

